In [4]:
using GynC
using JLD

In [ ]:
include("../scripts/gyncplots.jl");

In [ ]:
h=0.5
niter=5000
grad = true

function optimize(m, r) 
  w0=GynC.uniformweights(m.xs|>length);
  if grad 
    GynC.mple(m, w0, niter, r, h)
  else
    [GynC.optimineq(m,r,w0)] 
  end
end

In [ ]:
function getsamples(thin=10000)
  ss = GynC.readsamples("/datanumerik/bzfsikor/gync/master");
  all = vcat([s.samples for s in ss]...);
  samples = all[1:thin:end,:];
  @show size(samples,1)
  samples
end

function getmodels(samples, zmult)
  @time m = GynC.gyncmodel([samples[i,:] for i=1:size(samples,1)], GynC.alldatas(), zmult = zmult)
  @time ms = GynC.smoothedmodel(m, zmult)
  n = length(m.xs)
  @time muni = GynC.gyncmodel(vcat(GynC.samplepi0(n*5, trajts), m.xs), m.datas, zmult=0);
  m, ms, muni
end


function getweights(m, ms, muni)
  ws = Dict()
  @time ws["uni"] = [inverseweights(muni.xs)]
  @time ws["MPLE"] = optimize(m, 100/101)
  @time ws["NPMLE"] = optimize(m, 0)
  @time ws["DS-MLE"] = optimize(ms, 0);
  ws
end

In [ ]:
m,ms,muni = getmodels(getsamples(), 50)
ws = getweights(m,ms,muni)

In [8]:
@save "paperplots/grad$(grad)h$(h)niter$(niter).jld" m ms muni ws

In [7]:
ws

Dict{Any,Any} with 4 entries:
  "MPLE"   => Any[[0.000377644,0.000377644,0.000377644,0.000377644,0.000377644,…
  "NPMLE"  => Any[[0.000377644,0.000377644,0.000377644,0.000377644,0.000377644,…
  "uni"    => Array{Float64,1}[[6.72914e-5,6.73158e-5,6.71313e-5,6.72068e-5,6.7…
  "DS-MLE" => Any[[0.000377644,0.000377644,0.000377644,0.000377644,0.000377644,…